In [13]:
import graphlab as gl
import numpy as np
import pandas as pd
import os
from pylab import *

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1492314475.log


This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


In [20]:
# input parameters
#=================================
classifier_list = ['LR']
#=================================

window_length_list = [200]
window_step = 50
feature_type_list = [1, 2, 3] #1: prefrontal ROI, #2: Back ROIs, #3: all ROIs
l2_penalty_list = np.logspace(-2, 1.5, num=15, endpoint=True, base=10.0, dtype=None).tolist()
l1_penalty = 0.0

for mouse_id in range(1,7):

    for win_ln_idx in range(len(window_length_list)): 
        window_length = window_length_list[win_ln_idx]
        
        ##========================= load features
        # print on screen: progress
        current_progress = ("\n========================================\nLoading features: mouse%02d, window_length%03d ...\n" \
                            % (mouse_id, window_length))
        print current_progress

        # load and organize feature matrix
        comb = lz_load_feature_matrix(mouse_id, window_length, window_step)       
        
        for classifier_idx in range(len(classifier_list)):
            classifier = classifier_list[classifier_idx]

            # initiate
            AC  = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))] 
            SE  = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))] 
            SP  = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))]
            AUC = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))]

            for feature_type_idx in range(len(feature_type_list)):
                feature_type = feature_type_list[feature_type_idx]

                ##========================= classification
                for l2_penalty_idx in range(len(l2_penalty_list)):
                    l2_penalty = l2_penalty_list[l2_penalty_idx]

                    # print on screen: progress
                    current_progress = ("\n----------------------------------------\nClassifying --> classifier: %s, feature_type%03d, l2_penalty_%f ..." \
                                        % (classifier, feature_type, l2_penalty))
                    print current_progress

                    if classifier == 'LR':
                        AC[feature_type_idx][l2_penalty_idx],SE[feature_type_idx][l2_penalty_idx],SP[feature_type_idx][l2_penalty_idx],AUC[feature_type_idx][l2_penalty_idx] = \
                            lz_logistic_AC_SE_SP(comb, feature_type, l2_penalty, l1_penalty)

            ##====================== save results
            AC_saveName = ("%s_vg_mouse%02d_window_length%03d_AC.csv" \
                           % (classifier,mouse_id,window_length))
            SE_saveName = ("%s_vg_mouse%02d_window_length%03d_SE.csv" \
                           % (classifier,mouse_id,window_length))
            SP_saveName = ("%s_vg_mouse%02d_window_length%03d_SP.csv" \
                           % (classifier,mouse_id,window_length))
            AUC_saveName = ("%s_vg_mouse%02d_window_length%03d_AUC.csv" \
                           % (classifier,mouse_id,window_length))
            df_AC = pd.DataFrame(AC)
            df_SE = pd.DataFrame(SE)        
            df_SP = pd.DataFrame(SP)  
            df_AUC = pd.DataFrame(AUC)  
            
            os.chdir("/Users/lizhu/Dropbox/projects/calcium/comparison/ROI4FluoroSNNAP/classification_results") 
            df_AC.to_csv(AC_saveName, index = False, header = False)
            df_SE.to_csv(SE_saveName, index = False, header = False)
            df_SP.to_csv(SP_saveName, index = False, header = False)
            df_AUC.to_csv(AUC_saveName, index = False, header = False)


Loading features: mouse01, window_length200 ...


----------------------------------------
Classifying --> classifier: LR, feature_type001, l2_penalty_0.010000 ...
AC_mean =  0.883435408062
AC_std =  0.0331792292146
SE_mean =  0.651581247823
SE_std =  0.117619097111
SP_mean =  0.941135937884
SP_std =  0.025842352369

----------------------------------------
Classifying --> classifier: LR, feature_type001, l2_penalty_0.017783 ...
AC_mean =  0.883428286569
AC_std =  0.0267684480946
SE_mean =  0.655995326498
SE_std =  0.108688799413
SP_mean =  0.940204430336
SP_std =  0.0231207250633

----------------------------------------
Classifying --> classifier: LR, feature_type001, l2_penalty_0.031623 ...
AC_mean =  0.885956416465
AC_std =  0.0247011762838
SE_mean =  0.66378753429
SE_std =  0.108155942645
SP_mean =  0.941268260124
SP_std =  0.0236390517256

----------------------------------------
Classifying --> classifier: LR, feature_type001, l2_penalty_0.056234 ...
AC_mean =  0.885101837345
A

In [17]:
# load and organize feature matrix
def lz_load_feature_matrix(mouse_id, window_length, window_step):
    fileName = "format4ML_GC6f_emx_0" + str(mouse_id) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
    loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName
    comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
    colName_dg = 'degree'
    colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
    colName_cc = 'clusteringCoef'
    colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
    colName_pl = 'pathlength'
    colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
    colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
    colName = (list(colName))
    dictionary = dict(zip(comb.column_names(), colName))
    comb = comb.rename(dictionary)
    comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
    # comb['Whisker'].show(view = 'Categorical')
    
    return comb

In [14]:
# train and cross-validation
def lz_logistic_AC_SE_SP(data, feature_type, l2_penalty, l1_penalty):
    
    # clearify features

    if feature_type == 1:   
        ROI_list = [4, 5, 6, 7, 8, 11, 12, 13, 14, 15]

    elif feature_type == 2: 
        ROI_list = [1, 2, 16, 17, 18, 23, 24, 25, 26, 28]
        
    elif feature_type == 3: 
        ROI_list = range(0,30)
                
    ROI_dg = [x - 1 for x in ROI_list]
    ROI_cc = [x + 30 for x in ROI_dg]
    ROI_pl = [x + 60 for x in ROI_dg]
    
    feature_spk = list(comb.column_names()[i] for i in ROI_dg + ROI_cc + ROI_pl)
    
    feature = feature_spk

    # Kfold
    num_fold = 10
    folds = gl.cross_validation.KFold(comb, num_fold)
    SE = [None] * num_fold
    SP = [None] * num_fold
    AC = [None] * num_fold
    AUC = [None] * num_fold
    
    # print specificity
    idx = 0
    for train, valid in folds:
        m = gl.logistic_classifier.create(train,
                                          target='Whisker',
                                          features=feature,
                                          l2_penalty = l2_penalty, 
                                          l1_penalty = l1_penalty,
                                          validation_set=None,
                                          verbose = False)
        results = m.evaluate(valid)
        confusion_matrix = results['confusion_matrix']

        AUC[idx] = results['auc']
                    
        TP, TN, FP, FN = lz_extract_ACC_SE_SP(confusion_matrix)
        SP[idx] = float(TN) / (TN + FP)
        SE[idx] = float(TP) / (TP + FN)
        AC[idx] = float(TP+TN) / (TP+TN+FP+FN)
        
        idx = idx + 1
        
    AC_mn = np.mean(AC)
    AC_sd = np.std(AC)
    SE_mn = np.mean(SE)
    SE_sd = np.std(SE)
    SP_mn = np.mean(SP)
    SP_sd = np.std(SP)
    AUC_mn = np.mean(AUC)
    AUC_sd = np.std(AUC)
    
    print 'AC_mean = ', AC_mn
    print 'AC_std = ',  AC_sd
    print 'SE_mean = ', SE_mn
    print 'SE_std = ',  SE_sd
    print 'SP_mean = ', SP_mn
    print 'SP_std = ',  SP_sd
    
    return AC_mn, SE_mn, SP_mn, AUC_mn

In [15]:
def lz_extract_ACC_SE_SP(confusion_matrix):
    TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
    if np.size(TP) == 0:
        TP = 0
    else:
        TP = TP['count'][0]
    TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
    if np.size(TN) == 0:
        TN = 0
    else:
        TN = TN['count'][0]
    FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
    if np.size(FP) == 0:
        FP = 0
    else:
        FP = FP['count'][0]
    FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
    if np.size(FN) == 0:
        FN = 0
    else:
        FN = FN['count'][0]
    
    return TP, TN, FP, FN